# 데이터셋 구성하기  

목적:  
task에 fine-tune된 이진 분류 bert 모델 만들기 (by contrastive loss - BPR loss)

만들 것:  
1. user별 선호 item title들 serialize (일관성을 위해 텍스트 정렬)  
    ex: f'{item_title1} [SEP] {item_title2} [SEP] {item_title3} [SEP] {item_title4} [SEP] {item_title5} ...'  
    추후: serialize된 것들을 일부 분할해서 증강  

2. item별 보유 KG triplet의 serialize  
    ex: f'{item_title} [SEP] {relation1} [SEP] {attribute1} [SEP] {relation2} [SEP] {attribute2} [SEP] {relation3} [SEP] {attribute3} ...'  
    OR  
    f'{item_title} has relation {relation1} with attribute {attribute1}, relation {relation2} with attribute {attribute2}, relation {relation3} with attribute {attribute3}, ...'  

train - valid - test split 방식:  
train/valid/test: unique user의 0.8/0.1/0.1로 고정  
train/valid/test별 X와 y split 방식:  
X: train/valid/test별 0.5  
y: train/valid/test별 0.5 (나머지)  

user 중에서 interaction이 15개 이상인 경우만 활용 (55375 -> 53880)  

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/ml-100k_tmdb_final.csv")

In [10]:
pd.Index(list('abc')).get_loc('b')

1

In [166]:
# implict feedback으로 변경하기
def convert_rating(x):
    return 1 if x >= 4 else 0

df["rating"] = df["rating"].apply(lambda x:convert_rating(x))

In [167]:
df = df[df["rating"] == 1]
df.drop(labels=["rating"], axis=1, inplace=True)

/tmp/ipykernel_26951/1696300468.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(labels=["rating"], axis=1, inplace=True)


In [168]:
df

,userId,movieId,title,release_year,genre_list,origin_country,writer,director,cast
2,226,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
4,306,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
5,296,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
6,34,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
7,271,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
...,...,...,...,...,...,...,...,...,...
99983,655,1636,Brothers in Trouble,1990's,Drama,NaN,Robert Buckler,Udayan Prasad,Bhasker Patel
99989,532,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99990,416,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99992,655,1645,"Butcher Boy, The",1990's,Drama,NaN,NaN,NaN,NaN


1. user별 선호 item title들 serialize (일관성을 위해 텍스트 정렬)  
    ex: f'{item_title1} [SEP] {item_title2} [SEP] {item_title3} [SEP] {item_title4} [SEP] {item_title5} ...'  
    추후: serialize된 것들을 일부 분할해서 증강  

In [169]:
df.sort_values(by=["userId", "movieId"])

,userId,movieId,title,release_year,genre_list,origin_country,writer,director,cast
3526,1,1,Toy Story,1990's,Animation|Children's|Comedy,US,NaN,John Lasseter,Tom Hanks
35750,1,3,Four Rooms,1990's,Thriller,US,Quentin Tarantino,Quentin Tarantino,Salma Hayek Pinault
94342,1,6,Shanghai Triad (Yao a yao yao dao waipo qiao),1990's,Drama,NaN,NaN,NaN,NaN
32649,1,7,Twelve Monkeys,1990's,Drama|Sci-Fi,US,NaN,Terry Gilliam,Bruce Willis
65174,1,9,Dead Man Walking,1990's,Drama,NaN,Tim Robbins,Tim Robbins,Jack Black
...,...,...,...,...,...,...,...,...,...
54671,943,824,"Great White Hype, The",1990's,Comedy,NaN,NaN,NaN,NaN
83485,943,840,Last Man Standing,1990's,Action|Drama|Western,US,NaN,Walter Hill,Christopher Walken
91679,943,928,"Craft, The",1990's,Drama|Horror,NaN,NaN,NaN,NaN
54518,943,943,Killing Zoe,1990's,Thriller,NaN,Roger Avary,Roger Avary,Eric Stoltz


In [170]:
df = df.groupby("userId").filter(lambda x: len(x) >= 15)

In [171]:
df

,userId,movieId,title,release_year,genre_list,origin_country,writer,director,cast
2,226,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
4,306,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
5,296,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
6,34,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
7,271,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
...,...,...,...,...,...,...,...,...,...
99983,655,1636,Brothers in Trouble,1990's,Drama,NaN,Robert Buckler,Udayan Prasad,Bhasker Patel
99989,532,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99990,416,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99992,655,1645,"Butcher Boy, The",1990's,Drama,NaN,NaN,NaN,NaN


In [172]:
from collections import defaultdict

u_m = defaultdict(list)

for u, m in zip(df["userId"].to_list(), df["movieId"].to_list()):
    u_m[u].append(m)

In [173]:
m_t = defaultdict(str)

for m, t in zip(df["movieId"].to_list(), df["title"].to_list()):
    m_t[m] = t

In [174]:
u_tlist = defaultdict(list)

for u, m in u_m.items():
    for mid in m:
        u_tlist[u].append(m_t[mid])

In [175]:
import random

In [176]:
train_user = random.sample(list(u_tlist.keys()), int(len(list(u_tlist.keys())) * 0.8))

In [177]:
tmp = list(set(list(u_tlist.keys())) - set(train_user))

In [178]:
valid_user = random.sample(tmp, int(len(tmp) * 0.5))
test_user = list(set(tmp) - set(valid_user))

In [179]:
train_X = defaultdict(list)
train_y = defaultdict(list)
valid_X = defaultdict(list)
valid_y = defaultdict(list)
test_X = defaultdict(list)
test_y = defaultdict(list)

In [180]:
import numpy as np
from copy import deepcopy

def set_dataset(user_list:list, user_itemTitle_list:dict, user_itemId_list:dict, X:dict, y:dict):
    X_itemId = deepcopy(X)
    y_itemId = deepcopy(y)
    for u in user_list:
        u_item_list = np.array(user_itemTitle_list[u])
        u_itemId_list = np.array(user_itemId_list[u])
        if len(u_item_list) != len(u_itemId_list):
            raise Exception("두 리스트 길이 안 맞음. 있을 수 없는 상황.")
        shuffle_index = np.arange(len(u_item_list))
        np.random.shuffle(shuffle_index)
        u_item_list = u_item_list[shuffle_index]
        u_itemId_list = u_itemId_list[shuffle_index]
        
        sep = int(len(u_item_list) * 0.5)
        X[u] = u_item_list[:sep].tolist()
        y[u] = u_item_list[sep:].tolist()
        X_itemId[u] = u_itemId_list[:sep].tolist()
        y_itemId[u] = u_itemId_list[sep:].tolist()
    return X, y, X_itemId, y_itemId

In [181]:
train_X, train_y, train_X_itemId, train_y_itemId = set_dataset(train_user, u_tlist, u_m, train_X, train_y)

In [182]:
valid_X, valid_y, valid_X_itemId, valid_y_itemId = set_dataset(valid_user, u_tlist, u_m, valid_X, valid_y)

In [183]:
test_X, test_y, test_X_itemId, test_y_itemId = set_dataset(test_user, u_tlist, u_m, test_X, test_y)

In [184]:
train_dataset = [train_X, train_y, train_X_itemId, train_y_itemId]
valid_dataset = [valid_X, valid_y, valid_X_itemId, valid_y_itemId]
test_dataset = [test_X, test_y, test_X_itemId, test_y_itemId]

In [185]:
dataset = [train_dataset, valid_dataset, test_dataset]

In [186]:
import pickle

with open("user_dataset.pickle", "wb") as fw:
    pickle.dump(dataset, fw)

2. item별 보유 KG triplet의 serialize  
    ex: f'{item_title} [SEP] {relation1} [SEP] {attribute1} [SEP] {relation2} [SEP] {attribute2} [SEP] {relation3} [SEP] {attribute3} ...'  
    OR  
    f'{item_title} has relation {relation1} with attribute {attribute1}, relation {relation2} with attribute {attribute2}, relation {relation3} with attribute {attribute3}, ...'  

In [187]:
df.columns

Index(['userId', 'movieId', 'title', 'release_year', 'genre_list',
       'origin_country', 'writer', 'director', 'cast'],
      dtype='object')

In [188]:
df

,userId,movieId,title,release_year,genre_list,origin_country,writer,director,cast
2,226,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
4,306,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
5,296,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
6,34,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
7,271,242,Kolya,1990's,Comedy,CZ,NaN,Jan Svěrák,Silvia Šuvadová
...,...,...,...,...,...,...,...,...,...
99983,655,1636,Brothers in Trouble,1990's,Drama,NaN,Robert Buckler,Udayan Prasad,Bhasker Patel
99989,532,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99990,416,1594,Everest,1990's,Documentary,US,Tim Cahill,David Breashears,Liam Neeson
99992,655,1645,"Butcher Boy, The",1990's,Drama,NaN,NaN,NaN,NaN


In [189]:
df.drop_duplicates(subset=["movieId"]).drop(labels="userId", axis=1).sort_values(by="movieId", ascending=True).reset_index().drop(labels="index", axis=1)

,movieId,title,release_year,genre_list,origin_country,writer,director,cast
0,1,Toy Story,1990's,Animation|Children's|Comedy,US,NaN,John Lasseter,Tom Hanks
1,2,GoldenEye,1990's,Action|Adventure|Thriller,GB,NaN,Martin Campbell,Famke Janssen
2,3,Four Rooms,1990's,Thriller,US,Quentin Tarantino,Quentin Tarantino,Salma Hayek Pinault
3,4,Get Shorty,1990's,Action|Comedy|Drama,US,NaN,Barry Sonnenfeld,Harvey Keitel
4,5,Copycat,1990's,Crime|Drama|Thriller,US,NaN,Jon Amiel,Sigourney Weaver
...,...,...,...,...,...,...,...,...
1440,1656,Little City,1990's,Comedy|Romance,NaN,Roberto Benabib,Roberto Benabib,Annabella Sciorra
1441,1658,"Substance of Fire, The",1990's,Drama,NaN,NaN,NaN,NaN
1442,1662,Rough Magic,1990's,Drama|Romance,GB,NaN,Clare Peploe,Russell Crowe
1443,1664,8 Heads in a Duffel Bag,1990's,Comedy,NaN,Tom Schulman,Tom Schulman,George Hamilton


In [190]:
item_info = df.drop_duplicates(subset=["movieId"]).drop(labels="userId", axis=1).sort_values(by="movieId", ascending=True).reset_index().drop(labels="index", axis=1)

In [191]:
from collections import defaultdict

kg_triplet_serialize = defaultdict(list)

In [192]:
dic = item_info.to_dict()

In [193]:
dic.keys()

dict_keys(['movieId', 'title', 'release_year', 'genre_list', 'origin_country', 'writer', 'director', 'cast'])

In [194]:
for m, t, r_y, g, o, w, d, c in zip(dic['movieId'].values(), dic['title'].values(), dic['release_year'].values(), dic["genre_list"].values(), dic["origin_country"].values(), dic['writer'].values(), dic['director'].values(), dic['cast'].values()):
    kg_triplet_serialize[m] = [m, t, r_y, g, o, w, d, c]

In [195]:
for k, v in kg_triplet_serialize.items():
    print(k)
    print(v)
    break

1
[1, 'Toy Story', "1990's", "Animation|Children's|Comedy", 'US', nan, 'John Lasseter', 'Tom Hanks']


In [196]:
kg_triplet_dict = defaultdict(str)

# datasets 모듈 쓸 때 ### -> [SEP] 토큰으로 바꿔야함
# relation_dict = {2:" [SEP] RELATION released year ATTRIBUTE ", 3:" RELATION genres ATTRIBUTE ", 4:" RELATION origin country ATTRIBUTE ", 5:" RELATION written by ATTRIBUTE ", 6:" RELATION directed by ATTRIBUTE ", 7:" RELATION acted by ATTRIBUTE "}
relation_dict = {2:" [SEP] released year is ", 3:", genres are consisted as ", 4:", origin country is ", 5:", written by ", 6:", directed by ", 7:", acted by "}

for k, v in kg_triplet_serialize.items():
    triplet = ''
    if k == 0:
        continue
    else:
        for i, val in enumerate(v[1:]):
            if i + 1 == 1 and isinstance(val, str):
                triplet += "[CLS] " + val
            else:
                if isinstance(val, str):
                    triplet += relation_dict[i + 1]
                    if i + 1 == 3: # 장르
                        val = " ".join(val.split("|"))
                    triplet += val
                    if i == 6:
                        triplet += " [SEP]"
        if triplet == "":
            print(v)
        kg_triplet_dict[v[0]] = triplet
        # print(triplet)
    # if k == 10:
    #     break

In [197]:
kg_triplet_dataset = kg_triplet_dict

In [198]:
import pickle

with open("item_triplet_dataset.pickle", "wb") as fw:
    pickle.dump(kg_triplet_dataset, fw)

In [199]:
import pickle

with open("item_triplet_dataset.pickle", "rb") as fr:
    kg_triplet_dataset = pickle.load(fr)

In [200]:
for k, v in kg_triplet_dataset.items():
    if v == "":
        print(k)

In [201]:
kg_triplet_dataset

defaultdict(str,
            {1: "[CLS] Toy Story [SEP] released year is 1990's, genres are consisted as Animation Children's Comedy, origin country is US, directed by John Lasseter, acted by Tom Hanks [SEP]",
             2: "[CLS] GoldenEye [SEP] released year is 1990's, genres are consisted as Action Adventure Thriller, origin country is GB, directed by Martin Campbell, acted by Famke Janssen [SEP]",
             3: "[CLS] Four Rooms [SEP] released year is 1990's, genres are consisted as Thriller, origin country is US, written by Quentin Tarantino, directed by Quentin Tarantino, acted by Salma Hayek Pinault [SEP]",
             4: "[CLS] Get Shorty [SEP] released year is 1990's, genres are consisted as Action Comedy Drama, origin country is US, directed by Barry Sonnenfeld, acted by Harvey Keitel [SEP]",
             5: "[CLS] Copycat [SEP] released year is 1990's, genres are consisted as Crime Drama Thriller, origin country is US, directed by Jon Amiel, acted by Sigourney Weaver [S